In [12]:
%pip install flask transformers textblob spacy torch elasticsearch flask-testing


Note: you may need to restart the kernel to use updated packages.


In [13]:
import unittest
from flask_testing import TestCase
import json
import numpy as np
import torch
import torch.optim as optim
from collections import deque

# Import your Flask app from main.py
from main import app, understand_query, select_action, optimize_model, WebScrapingEnv

# Ensure that your necessary classes and functions are imported
# For example, QNetwork, WebScrapingEnv, etc.


In [14]:
class FlaskRoutesTestCase(TestCase):
    def create_app(self):
        return app

    def test_webscrape_valid_input(self):
        response = self.client.post('/api/webscrape', json={'input': 'example query'})
        self.assert200(response)
        data = json.loads(response.data)
        self.assertIn('best_result', data)

    def test_webscrape_no_input(self):
        response = self.client.post('/api/webscrape', json={})
        self.assert400(response)
        data = json.loads(response.data)
        self.assertEqual(data['detail'], 'No input provided')

    def test_search_elasticsearch(self):
        response = self.client.get('/api/search', query_string={'query': 'example query'})
        self.assert200(response)
        data = json.loads(response.data)
        self.assertIn('results', data)

    def test_feedback(self):
        response = self.client.post('/api/feedback', json={'feedback': 1.0})
        self.assert200(response)
        data = json.loads(response.data)
        self.assertEqual(data['message'], 'Feedback received')

    # Additional test cases for error handling
    def test_webscrape_invalid_input(self):
        response = self.client.post('/api/webscrape', json={'input': 12345})
        self.assert400(response)
        data = json.loads(response.data)
        self.assertEqual(data['detail'], 'Invalid input type')

    def test_search_no_query(self):
        response = self.client.get('/api/search', query_string={})
        self.assert400(response)
        data = json.loads(response.data)
        self.assertEqual(data['detail'], 'Query parameter missing')

    def test_feedback_invalid_feedback(self):
        response = self.client.post('/api/feedback', json={'feedback': 'not_a_number'})
        self.assert400(response)
        data = json.loads(response.data)
        self.assertEqual(data['detail'], 'Invalid feedback value')


In [15]:
class BusinessLogicTestCase(unittest.TestCase):
    def test_understand_query(self):
        result = understand_query("Barack Obama was the president of the USA.")
        self.assertIn(('Barack Obama', 'PERSON'), result)
        self.assertIn(('USA', 'GPE'), result)

    def test_understand_query_empty(self):
        result = understand_query("")
        self.assertEqual(result, [])

    def test_select_action(self):
        q_network = QNetwork()  # Ensure QNetwork is imported or defined
        state = np.array([0.0])
        action = select_action(state, epsilon=0.5, q_network=q_network)
        self.assertIn(action, [0, 1])

    def test_optimize_model(self):
        q_network = QNetwork()  # Ensure QNetwork is imported or defined
        optimizer = optim.Adam(q_network.parameters(), lr=1e-3)
        memory = deque(maxlen=2000)
        state = np.array([0.0])
        action = 0
        reward = 1.0
        next_state = np.array([1.0])
        done = False
        memory.append((state, action, reward, next_state, done))
        optimize_model(memory, q_network, optimizer)

    def test_optimize_model_empty_memory(self):
        q_network = QNetwork()  # Ensure QNetwork is imported or defined
        optimizer = optim.Adam(q_network.parameters(), lr=1e-3)
        memory = deque(maxlen=2000)
        optimize_model(memory, q_network, optimizer)  # Test with empty memory

    def test_web_scraping_env(self):
        env = WebScrapingEnv()  # Ensure WebScrapingEnv is imported or defined
        state = env.reset()
        self.assertEqual(state, np.array([0.0], dtype=np.float32))
        next_state, reward, done, _ = env.step(0, {'feedback': None})
        self.assertEqual(next_state, np.array([1.0], dtype=np.float32))  # Assumes action 0 will change state
        self.assertFalse(done)

    def test_web_scraping_env_invalid_action(self):
        env = WebScrapingEnv()  # Ensure WebScrapingEnv is imported or defined
        state = env.reset()
        with self.assertRaises(ValueError):
            env.step(-1, {'feedback': None})  # Test with invalid action


In [16]:
# Run Flask route tests
print("Running Flask route tests:")
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(FlaskRoutesTestCase))

# Run business logic tests
print("Running business logic tests:")
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(BusinessLogicTestCase))


.F

Running Flask route tests:


2024-08-18 11:51:16,203 - DEBUG - Starting new HTTP connection (1): localhost:9200
2024-08-18 11:51:16,209 - INFO - POST http://localhost:9200/web_scraping/_search [status:N/A duration:0.006s]
2024-08-18 11:51:16,211 - WARNING - Node <Urllib3HttpNode(http://localhost:9200)> has failed for 1 times in a row, putting on 1 second timeout
2024-08-18 11:51:16,216 - WARNING - Retrying request after failure (attempt 0 of 3)
Traceback (most recent call last):
  File "/home/lychan23/Desktop/Ly.JSxPY/deps/lib/python3.10/site-packages/elastic_transport/_transport.py", line 342, in perform_request
    resp = node.perform_request(
  File "/home/lychan23/Desktop/Ly.JSxPY/deps/lib/python3.10/site-packages/elastic_transport/_node/_http_urllib3.py", line 202, in perform_request
    raise err from None
elastic_transport.ConnectionError: Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x724654ad5960>: Failed to establish a new connection: [Errno 111] Connection 

Running business logic tests:


.E
ERROR: test_optimize_model (__main__.BusinessLogicTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_19542/566571651.py", line 18, in test_optimize_model
    q_network = QNetwork()  # Ensure QNetwork is imported or defined
NameError: name 'QNetwork' is not defined

ERROR: test_optimize_model_empty_memory (__main__.BusinessLogicTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_19542/566571651.py", line 30, in test_optimize_model_empty_memory
    q_network = QNetwork()  # Ensure QNetwork is imported or defined
NameError: name 'QNetwork' is not defined

ERROR: test_select_action (__main__.BusinessLogicTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_19542/566571651.py", line 12, in test_select_action
    q_network = QNetwor

<unittest.runner.TextTestResult run=7 errors=4 failures=0>